In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Modeling

- 참고) 이미 모델별 성능 비교나 하이퍼파라미터 탐색은 끝난거니까 성능평가나 시각화는 다 뺐어. 전체적인 흐름은 사용자가 미세먼지, 기온 데이터만 입력하면 자동 인코딩, 스케일링해서 해당 지역의 평균 데이터에 합쳐 데이터프레임 형식으로 변환한다음에 세 가지 모델이 예측한 값들을 앙상블하는거야. 그 다음에는 다시 사용자에게 발전량을 보여줘야 하니까 발전량 reverse scaled 된 값을 출력하는거야.
- 일단 날짜만 입력받는다고 가정하고 모델 만들어봤어.
- 날짜를 입력받는 형식은 yyyy-mm-dd.
- 그리고 지난 회의에서 학습된 모델로 예측할 때, 학습한 데이터 범위 밖의 데이터가 들어올 경우 오류가 생길 수 있다고 했잖아? 찾아보니
https://www.inflearn.com/questions/54854
여기에 잘 나와있더라고~ 참고해!
- 그래서 내가 생각해본건 모델 구축할 때부터 year를 빼는거야. 그럼 미래의 어떤 날짜를 입력하더라도 연도만 뺀다면 달과 일은 학습 범위내에 있을거잖아? 일단은 내 생각대로 해봤고 더 좋은 생각있으면 공유해줘~

In [ ]:
# 데이터 인코딩, 스케일링 후 훈련, 테스트 데이터셋으로 분류
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy import stats
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
from lightgbm import LGBMRegressor
import lightgbm as lgb
from sklearn.linear_model import LinearRegression, Ridge, Lasso
import math

data = pd.read_csv("/content/drive/MyDrive/태양광 프로젝트!/merged_final.csv")

#Label Encoding
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le = le.fit(data['area'])
data['area2'] = le.transform(data['area'])
data['area2'] = data['area2'].astype('category')

data['day'] = data['date'].str.split('-').str[2]
data['month'] = data['date'].str.split('-').str[1]
data['year'] = data['date'].str.split('-').str[0]

data.drop(['area', 'date'], axis = 1, inplace = True)

#발전량 이상치 제거 & 정규화
data = data[data['generation'] != 0]

#이상치라고 생각되는 값만 빼고 추출
data = data[(data['generation'] < 3000) & (data['generation'] >1)]
data = data[data['avgWs'] < 6]
data = data[data['PM10'] < 150]
data = data[data['PM25'] < 80]

data['day'] = data['day'].astype(float)
data['year'] = data['year'].astype(float)
data['month'] = data['month'].astype(float)
data['area2'] = data['area2'].astype(float)
data = data.reset_index(drop = True)

#Encoding에 앞서 변수 분리
y_target = data['generation']
X_features1 = data.drop(['generation', 'year', 'day', 'area2', 'month'], axis = 1, inplace = False)
X_features2 = data[['month', 'day', 'area2']]

#지역, 날짜 변수는 one-hot encoding, 그 외의 변수는 min-max scaler 사용
#min-max scaler
#발전량, 지역, 날짜 변수 제외 min-max scaling
from sklearn.preprocessing import MinMaxScaler
scaler_ = MinMaxScaler()
scaler_.fit(X_features1)
X_scaled = scaler_.transform(X_features1)
X_features = pd.DataFrame(X_scaled, columns = X_features1.columns)
#날짜, 지역 one-hot encoding
X_features_ohe =pd.get_dummies(X_features2, columns=['month', 'day', 'area2'])
X_features_ = pd.concat([X_features, X_features_ohe], axis = 1)
#발전량 로그 정규화
y_target_log = np.log1p(y_target)
# 7:3으로 훈련, 테스트 데이터셋 나누기
X_train, X_test, y_train, y_test = train_test_split(X_features_, y_target_log, test_size=0.3, random_state=1234)

In [ ]:
# RIDGE
ridge_reg = Ridge()
ridge_reg = Ridge(alpha=0.05)
ridge_reg.fit(X_train, y_train)

# LGBM
from sklearn.model_selection import KFold
folds = 5
kf = KFold(n_splits = folds)

lgbm_params = {
    'objective' : 'regression',
    'n_estimators' : 500,
    'num_leaves' : 64,
    'max_depth' : 160,
    'learning_rate' : 0.05,
    'min_child_samples' : 120,
    'subsample' : 0.8,
    'random_seed' : 1234
}
train_X = X_features_
train_Y = y_target_log
models = []
oof = np.zeros(len(train_X))

for train_index, val_index in kf.split(train_X):
  X_train = train_X.iloc[train_index]
  X_valid = train_X.iloc[val_index]
  y_train = train_Y.iloc[train_index]
  y_valid = train_Y.iloc[val_index]

  lgb_train = lgb.Dataset(X_train, y_train)
  lgb_eval = lgb.Dataset(X_valid, y_valid, reference = lgb_train)

  model_lgb = lgb.train(lgbm_params,
                        lgb_train,
                        valid_sets = lgb_eval, 
                        num_boost_round = 100,
                        early_stopping_rounds = 20,
                        verbose_eval = 10,)
  
  y_pred = model_lgb.predict(X_valid, num_iteration = model_lgb.best_iteration)
  models.append(model_lgb)
  oof[val_index] = y_pred

#RANDOM FOREST
from sklearn.ensemble import RandomForestRegressor as rf
train_x = X_features_
train_y = y_target_log

models_rf = []
oof_rf = np.zeros(len(train_x))
for train_index, val_index in kf.split(train_x):
  X_train = train_x.iloc[train_index]
  X_valid = train_x.iloc[val_index]
  y_train = train_y.iloc[train_index]
  y_valid = train_y.iloc[val_index]
  model_rf = rf(
      n_estimators = 50,
      max_features = 6,
      min_samples_leaf = 7,
      random_state = 1234
  )
  model_rf.fit(X_train, y_train)
  y_pred = model_rf.predict(X_valid)  
  models_rf.append(model_rf)
  oof_rf[val_index] = y_pred

/usr/local/lib/python3.7/dist-packages/lightgbm/engine.py:118: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


Training until validation scores don't improve for 20 rounds.
[10]	valid_0's l2: 2.89099
[20]	valid_0's l2: 3.88662
Early stopping, best iteration is:
[1]	valid_0's l2: 1.90582
Training until validation scores don't improve for 20 rounds.
[10]	valid_0's l2: 1.50116
[20]	valid_0's l2: 1.50326
[30]	valid_0's l2: 1.58752
Early stopping, best iteration is:
[15]	valid_0's l2: 1.48129
Training until validation scores don't improve for 20 rounds.
[10]	valid_0's l2: 3.22501
[20]	valid_0's l2: 3.28047
Early stopping, best iteration is:
[3]	valid_0's l2: 3.18354
Training until validation scores don't improve for 20 rounds.
[10]	valid_0's l2: 3.24295
[20]	valid_0's l2: 3.62244
Early stopping, best iteration is:
[1]	valid_0's l2: 3.00624
Training until validation scores don't improve for 20 rounds.
[10]	valid_0's l2: 3.54674
[20]	valid_0's l2: 4.54254
Early stopping, best iteration is:
[1]	valid_0's l2: 2.50861


# 사용자 입력값 인코딩 모듈

In [ ]:
#사용자가 클릭한 버튼의 encoded number가 area2 = 1.0이라고 가정해봄 
area2_input = 1.0

#버튼으로 지역 클릭 -> if문으로 area2 = 에 해당하는 float형 숫자 입력
date_input = input('예상 발전량이 궁금한 날짜를 입력해주세요 : ').split('-')
#입력된 날짜를 달과 일로 split한 뒤 밑의 코드에 대입
year_input = int(date_input[0])
month_input = float(date_input[1])
day_input = float(date_input[2])
# 사용자가 입력한 지역과 월의 평균 발전량
monthly = data.loc[(data['area2'] == area2_input) & (data['month'] == month_input)]
avg_mon = monthly['generation'].mean()

# 사용자가 입력한 지역과 날짜의 예측 발전량
temp_input = data.loc[(data['area2'] == area2_input) & (data['day'] == day_input) & (data['month'] == month_input)]
temp_col = ['avgTa', 'avgWs', 'avgRhm', 'ssDur', 'sumSsHr', 'sumGsr','avgTca', 'PM10', 'PM25']
avg_input = temp_input[temp_col].mean()

avg = pd.DataFrame(avg_input)
avg = avg.T

#입력받은 area, month, day 조합해 df만들기
w = np.array([area2_input, day_input, month_input])
w = w.reshape(1,-1)
weather_input = pd.DataFrame(w, columns = ['area2', 'day', 'month']) 

#전체 인코딩, 스케일링
avg_scaled = scaler_.transform(avg)
avg_feature = pd.DataFrame(avg_scaled, columns = avg.columns)

weather_df= pd.get_dummies(weather_input, columns = ['month', 'day', 'area2'])
result = pd.concat([avg_feature, weather_df], axis = 1)
#feature 수 맞춰주기 위해 X_features_의 행 1개를 가져와서 모두 0으로 입력한 뒤 concat
cc = X_features_.head(1)
cc[:] = 0
result = pd.concat([cc, result], axis = 0, ignore_index = True)
result = result.fillna(0)
result[:] = result[:].astype(float)

#PREDICTION
#result의 첫째 행은 0으로 이루어진 행이니까 result[1]로 예측값 가져오기
rid_p = ridge_reg.predict(result)[1] 
lgb_p = model_lgb.predict(result)[1]
rf_p = model_rf.predict(result)[1]
# rmsle수치로 가중치 계산 후 결과값 앙상블
# rmsle 수치가 작을 수록 성능이 더 좋다는 의미이니까 가중치를 더 크게 부여했음.
real = rid_p*0.5 + lgb_p*0.36 + rf_p*0.14
pred_gen = np.expm1(real)
print('%d월의 평균 태양광 발전량은 %.3f kW입니다.' %(month_input, avg_mon))
print('%d년 %d월 %d일의 태양광 예측 발전량은 %.3f kW입니다.' %(year_input, month_input, day_input, pred_gen))

예상 발전량이 궁금한 날짜를 입력해주세요 : 2023-06-07
6월의 평균 태양광 발전량은 503.708 kW입니다.
2023년 6월 7일의 태양광 예측 발전량은 254.594 kW입니다.


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


- rmsle 수치 /  가중치
- ridge 0.088 -> 0.50
- lgbm  0.24  -> 0.36
- rf    0.33  -> 0.14
- sum : 0.658

In [ ]:
#환경영향 환산
print('이 발전량은 경유차 %d대분 초미세먼지 저감효과와 같습니다.' %(math.trunc(pred_gen/4.2)))
print('이 발전량은 소나무 %d그루를 심은 것과 같습니다.' %(math.trunc(pred_gen/0.3*2.5)))
print('이 발전량은 온실가스 %.3fkg을 감축시킨 것과 같습니다.' %(pred_gen*0.46625))
print('이 발전량으로 %d원의 전기요금을 줄일 수 있습니다.' %(pred_gen*80))

이 발전량은 경유차 60대분 초미세먼지 저감효과와 같습니다.
이 발전량은 소나무 2121그루를 심은 것과 같습니다.
이 발전량은 온실가스 118.704kg을 감축시킨 것과 같습니다.
이 발전량으로 20367원의 전기요금을 줄일 수 있습니다.


In [ ]:
#['강원도' '경기도' '경상남도' '경상북도' '광주시' '대구시' '대전시' '부산시' '서울시' '세종시' '울산시' '인천시'
# '전라남도' '전라북도' '제주도' '충청남도' '충청북도']
# 0~16



- 경유차 1대당 연간 초미세먼지 발생량 0.602kg. 4.2kW당 경유차 1대분 초미세먼지 저감효과(https://www.sunnyseoul.com/citizen/miniPower.do)
- 0.3kW당 소나무 2.5그루 식재효과 
- 전력배출계수 : 0.46625tCO2eq/MWh(공공부문 온실가스, 에너지 목표관리 운영 등에 관한 지침에 근거)